In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import datetime as dt
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicting-employee-attrition/sample_submission_znWiLZ4.csv
/kaggle/input/predicting-employee-attrition/test_hXY9mYw.csv
/kaggle/input/predicting-employee-attrition/train_MpHjUjU.csv


In [2]:
train_df = pd.read_csv("../input/predicting-employee-attrition/train_MpHjUjU.csv")
test_df = pd.read_csv("../input/predicting-employee-attrition/test_hXY9mYw.csv")

print(f'Train Data Shape {train_df.shape}')
print(f'Test Data Shape {test_df.shape}')

Train Data Shape (19104, 13)
Test Data Shape (741, 1)


In [3]:
train_df.sample(5)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
10724,2016-11-01,1601,31,Male,C12,College,59932,2016-08-29,NaN,2,2,0,1
1959,2016-08-01,308,31,Male,C5,Master,188418,2011-08-15,NaN,2,5,937570,4
14972,2016-03-01,2219,33,Male,C25,Bachelor,110792,2014-12-19,NaN,2,3,2218640,3
5890,2016-10-01,882,30,Male,C13,College,94945,2010-10-12,NaN,2,4,0,1
19038,2017-01-01,2779,28,Male,C26,College,95133,2017-01-26,NaN,3,3,0,1


In [4]:
test_df.sample(5)

,Emp_ID
361,45
36,2420
162,112
385,889
496,1518


In [5]:
len(test_df['Emp_ID'].isin(train_df['Emp_ID'])) ### All the test employee id is present in train data frame

741

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   MMM-YY                19104 non-null  object
 1   Emp_ID                19104 non-null  int64 
 2   Age                   19104 non-null  int64 
 3   Gender                19104 non-null  object
 4   City                  19104 non-null  object
 5   Education_Level       19104 non-null  object
 6   Salary                19104 non-null  int64 
 7   Dateofjoining         19104 non-null  object
 8   LastWorkingDate       1616 non-null   object
 9   Joining Designation   19104 non-null  int64 
 10  Designation           19104 non-null  int64 
 11  Total Business Value  19104 non-null  int64 
 12  Quarterly Rating      19104 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 1.9+ MB


In [7]:
### Output variable with missing values
[col for col in train_df.columns if train_df[col].isna().sum()>0]

['LastWorkingDate']

In [8]:
### Get number of unique values in each columns
for col in train_df.columns:
    print('-'*60)
    print('Number of Unique Values in {} column is:'.format(col), train_df[col].nunique())

------------------------------------------------------------
Number of Unique Values in MMM-YY column is: 24
------------------------------------------------------------
Number of Unique Values in Emp_ID column is: 2381
------------------------------------------------------------
Number of Unique Values in Age column is: 36
------------------------------------------------------------
Number of Unique Values in Gender column is: 2
------------------------------------------------------------
Number of Unique Values in City column is: 29
------------------------------------------------------------
Number of Unique Values in Education_Level column is: 3
------------------------------------------------------------
Number of Unique Values in Salary column is: 2383
------------------------------------------------------------
Number of Unique Values in Dateofjoining column is: 869
------------------------------------------------------------
Number of Unique Values in LastWorkingDate column is:

##### *We can see the total number of unique employee is only 2381, meaning we have the duplicate records. It could be that an employee has grown within the company or as the data is by each month we can have some employee repeated each month, etc. We will test all this hypothesis later...*

In [9]:
for col in train_df.columns:
    print('***********{}************'.format(col))
    print(train_df[col].unique(), "\n")

***********MMM-YY************
['2016-01-01' '2016-02-01' '2016-03-01' '2017-11-01' '2017-12-01'
 '2016-12-01' '2017-01-01' '2017-02-01' '2017-03-01' '2017-04-01'
 '2017-08-01' '2017-09-01' '2017-10-01' '2016-07-01' '2016-08-01'
 '2016-09-01' '2016-10-01' '2016-11-01' '2016-04-01' '2016-05-01'
 '2016-06-01' '2017-05-01' '2017-06-01' '2017-07-01'] 

***********Emp_ID************
[   1    2    4 ... 2786 2787 2788] 

***********Age************
[28 31 43 29 34 35 30 39 42 27 26 33 40 41 32 22 44 36 21 49 37 38 46 47
 48 25 24 45 51 52 23 50 53 54 55 58] 

***********Gender************
['Male' 'Female'] 

***********City************
['C23' 'C7' 'C13' 'C9' 'C11' 'C2' 'C19' 'C26' 'C20' 'C17' 'C29' 'C10'
 'C24' 'C14' 'C6' 'C28' 'C5' 'C18' 'C27' 'C15' 'C8' 'C25' 'C21' 'C1' 'C4'
 'C3' 'C16' 'C22' 'C12'] 

***********Education_Level************
['Master' 'College' 'Bachelor'] 

***********Salary************
[57387 67016 65603 ... 35370 69498 70254] 

***********Dateofjoining************
['2015-12

##### Data Finding:
* Data does not contain any errored values.
* Only column with missing value is **LastWorkingDate** which makes sense as not all the employees has left the company

##### Data Preprocessing/ Feature Engineering:
* All the dates column are object type and hence need to change them to datetime
  - We can create some features from this date variables indicating the time-period
* Label Encoding is required for categorical variables
* We have few Ordinal variables so no need to do any encoding on them 

In [10]:
train_df['MMM-YY'].max(), train_df['MMM-YY'].min() 

('2017-12-01', '2016-01-01')

In [11]:
## Casting variable to their correct data type
# convert to datetime and parse day, month, year, and yymm from the user_created_date
train_df['LastWorkingDate'].fillna(0, inplace=True)
train_df['MMM-YY']  = pd.to_datetime(train_df['MMM-YY']).dt.tz_localize(None)
train_df['Dateofjoining']  = pd.to_datetime(train_df['Dateofjoining']).dt.tz_localize(None)
train_df['LastWorkingDate']  = pd.to_datetime(train_df['LastWorkingDate']).dt.tz_localize(None)

## Create Target variable - 'Attrition {0: current employee, 1: ex-employee}'
train_df['Attrition'] = np.where(train_df['LastWorkingDate']=='1970-01-01', 0, 1)

## Data Aggregation
Max_att_df = train_df.groupby(['Emp_ID','Gender','Education_Level','City', 'Attrition'], as_index=False) \
             ['Age','Salary','Joining Designation','Dateofjoining','Quarterly Rating','LastWorkingDate'].max()
Mode_att_df = train_df.groupby(['Emp_ID','Gender','Education_Level','City', 'Attrition'], as_index=False)['Total Business Value'].sum()

new_train_df0 = pd.merge(Max_att_df, Mode_att_df, on=['Emp_ID','Gender','Education_Level','City', 'Attrition'])

## Calculating Tenure (number of months since resignation)
new_train_df0['Tenure(M)'] = np.where(new_train_df0['LastWorkingDate']!='1970-01-01', 
                                   ((new_train_df0['LastWorkingDate'] - new_train_df0['Dateofjoining'])/np.timedelta64(1, 'M')).astype(int),
                                   ((pd.to_datetime('2017-12-31 23:59:59') - new_train_df0['Dateofjoining'])/np.timedelta64(1, 'M')).astype(int))
                                   

## Calcuating Days on Current Role
# temp = train_df[['MMM-YY', 'Emp_ID', 'Dateofjoining', 'Joining Designation', 'Designation']]
# temp['MonthsOnCurrentRole']=np.where(temp['Joining Designation']!=temp['Designation'], 
#                                   ((temp['MMM-YY'] - temp['Dateofjoining'])/np.timedelta64(1, 'M')).astype(int), 
#                                    0)
# temp1 = temp[temp['MonthsOnCurrentRole']>0]
# temp2 = temp1.groupby(['Emp_ID', 'Dateofjoining', 'Joining Designation', 'Designation'])['MonthsOnCurrentRole'].max().reset_index()

# # getting MonthOnCurrentRole to the original dataframe
# new_train_df1 = new_train_df0.merge(temp2, on=['Emp_ID', 'Dateofjoining', 'Joining Designation'])

new_train_df1 = new_train_df0.groupby(['Emp_ID', 'Gender', 'Education_Level', 'City'], as_index=False) \
                        ['Age', 'Salary', 'Quarterly Rating', 'Tenure(M)'].max()
print(new_train_df1.shape, new_train_df1.Emp_ID.nunique())

new_train_df2 = new_train_df0.groupby(['Emp_ID', 'Gender', 'Education_Level', 'City'], as_index=False) \
                        ['Attrition', 'Total Business Value'].sum()
print(new_train_df2.shape, new_train_df2.Emp_ID.nunique())

new_train_df3 = new_train_df2.merge(new_train_df1, on=['Emp_ID', 'Gender', 'Education_Level', 'City'])
print(new_train_df3.shape, new_train_df3.Emp_ID.nunique())
new_train_df3.head()

(2381, 8) 2381
(2381, 6) 2381
(2381, 10) 2381


,Emp_ID,Gender,Education_Level,City,Attrition,Total Business Value,Age,Salary,Quarterly Rating,Tenure(M)
0,1,Male,Master,C23,1,1715580,28,57387,2,24
1,2,Male,Master,C7,0,0,31,67016,1,1
2,4,Male,Master,C13,1,350000,43,65603,1,12
3,5,Male,College,C9,1,120360,29,46368,1,23
4,6,Female,Bachelor,C11,0,1265000,31,78728,2,5


In [12]:
new_train_df3[new_train_df3['Emp_ID']==2124]

,Emp_ID,Gender,Education_Level,City,Attrition,Total Business Value,Age,Salary,Quarterly Rating,Tenure(M)
1805,2124,Male,Bachelor,C13,1,3045460,23,68936,2,28


In [13]:
# Get frequency counts of categorical variables
cat_var = ['Gender', 'Education_Level', 'Attrition', 'Quarterly Rating', 'City']
for i in cat_var:
    print('-'*50, '\n', round(new_train_df3[i].value_counts() / len(new_train_df3) *100, 2))

-------------------------------------------------- 
 Male      58.97
Female    41.03
Name: Gender, dtype: float64
-------------------------------------------------- 
 Master      33.68
Bachelor    33.39
College     32.93
Name: Education_Level, dtype: float64
-------------------------------------------------- 
 1    67.87
0    32.13
Name: Attrition, dtype: float64
-------------------------------------------------- 
 1    50.48
2    20.33
3    14.91
4    14.28
Name: Quarterly Rating, dtype: float64
-------------------------------------------------- 
 C20    6.38
C15    4.24
C29    4.03
C26    3.91
C8     3.74
C27    3.74
C10    3.61
C16    3.53
C22    3.44
C3     3.44
C28    3.44
C12    3.40
C5     3.36
C1     3.36
C21    3.32
C14    3.32
C6     3.28
C4     3.23
C7     3.19
C9     3.15
C25    3.11
C23    3.11
C24    3.07
C19    3.02
C2     3.02
C17    2.98
C13    2.98
C18    2.90
C11    2.69
Name: City, dtype: float64


In [14]:
## Encoding Variables
encoded_df = new_train_df3.copy()
encoded_df['Gender'] = encoded_df['Gender'].map({'Male':0, 'Female':1})
edu_encoded_df = pd.get_dummies(encoded_df['Education_Level'])
city_encoded_df = pd.get_dummies(encoded_df['City'])

final_df0 = pd.concat([encoded_df, edu_encoded_df], axis=1)
final_df1 = pd.concat([final_df0, city_encoded_df], axis=1)

final_df1.drop(['Education_Level', 'City'], axis=1, inplace=True) #'LastWorkingDate', 'Dateofjoining'
# print(new_train_df0.shape)
print(final_df1.shape)
final_df1.head()

(2381, 40)


,Emp_ID,Gender,Attrition,Total Business Value,Age,Salary,Quarterly Rating,Tenure(M),Bachelor,College,...,C27,C28,C29,C3,C4,C5,C6,C7,C8,C9
0,1,0,1,1715580,28,57387,2,24,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,31,67016,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,0,1,350000,43,65603,1,12,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,1,120360,29,46368,1,23,0,1,...,0,0,0,0,0,0,0,0,0,1
4,6,1,0,1265000,31,78728,2,5,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
final_df1.select_dtypes(exclude=['int', 'uint8', 'int64']).columns

Index([], dtype='object')

In [16]:
# checking target variable distribution
round(final_df1.Attrition.value_counts() / len(final_df1) * 100, 2)

1    67.87
0    32.13
Name: Attrition, dtype: float64

In [17]:
# bringing test_df 
test_df1 = test_df.merge(final_df1, on=['Emp_ID'])
print(test_df1.shape, test_df1.Emp_ID.nunique())
print(test_df1.Attrition.value_counts())

test_df1.drop(['Attrition'], axis=1, inplace=True)

''' None of the employee has left the company (non attrition)'''
test_df1.head()

(741, 40) 741
0    741
Name: Attrition, dtype: int64


,Emp_ID,Gender,Total Business Value,Age,Salary,Quarterly Rating,Tenure(M),Bachelor,College,Master,...,C27,C28,C29,C3,C4,C5,C6,C7,C8,C9
0,394,1,27458220,34,97722,3,78,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,173,0,19866290,39,56174,4,78,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,1090,0,50382490,39,96750,4,78,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,840,1,17134040,40,88813,2,77,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,308,0,58024490,32,188418,4,76,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [18]:
final_train_df = final_df1[~final_df1['Emp_ID'].isin(test_df['Emp_ID'])]
print(final_train_df.shape, final_train_df.Emp_ID.nunique())
final_train_df.head()

(1640, 40) 1640


,Emp_ID,Gender,Attrition,Total Business Value,Age,Salary,Quarterly Rating,Tenure(M),Bachelor,College,...,C27,C28,C29,C3,C4,C5,C6,C7,C8,C9
0,1,0,1,1715580,28,57387,2,24,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,1,350000,43,65603,1,12,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,1,120360,29,46368,1,23,0,1,...,0,0,0,0,0,0,0,0,0,1
5,8,0,1,0,34,70656,1,3,0,1,...,0,0,0,0,0,0,0,0,0,0
7,12,0,1,2607180,35,28116,4,18,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# checking target variable distribution
round(final_train_df.Attrition.value_counts() / len(final_train_df) * 100, 2)

''' Need to apply SMOTE as target is highly imbalanced '''

' Need to apply SMOTE as target is highly imbalanced '

In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [21]:
X = final_train_df.drop(['Attrition', 'Emp_ID'], axis=1)
y = final_train_df[['Attrition']]

# Split data into train and test sets as well as for validation and testing
train, test, target_train, target_val = train_test_split(X, y, train_size= 0.85, random_state=0)

oversampler=SMOTE(random_state=0)
smote_train, smote_target = oversampler.fit_resample(train, target_train)

In [22]:
print('train', train.shape, 'test', test.shape, 'target_train', target_train.shape, 'target_val', target_val.shape )
smote_train.shape, smote_target.shape

train (1394, 38) test (246, 38) target_train (1394, 1) target_val (246, 1)


((2744, 38), (2744, 1))

#### RF Classifier

In [23]:
seed = 0   # We set our random seed to zero for reproducibility
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 1000,
#     'warm_start': True, 
    'max_features': 0.3,
    'max_depth': 4,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 0
}

In [24]:
rf = RandomForestClassifier() #
rf.fit(smote_train, smote_target)
print("Fitting of Random Forest finished")

rf_predictions = rf.predict(test)
print("Predictions finished")

''' Scoring the model '''
print("Accuracy score: {}".format(accuracy_score(target_val, rf_predictions)))
print("="*80)
print(classification_report(target_val, rf_predictions))

Fitting of Random Forest finished
Predictions finished
Accuracy score: 0.9878048780487805
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.99      1.00      0.99       244

    accuracy                           0.99       246
   macro avg       0.50      0.50      0.50       246
weighted avg       0.98      0.99      0.99       246



In [25]:
# Gradient Boosting Parameters
gb_params ={
    'n_estimators': 1500,
    'max_features': 0.9,
    'learning_rate' : 0.25,
    'max_depth': 4,
    'min_samples_leaf': 2,
    'subsample': 1,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 0
}

gb = GradientBoostingClassifier() #**gb_params
# Fit the model to our SMOTEd train and target
gb.fit(smote_train, smote_target)
# Get our predictions
gb_predictions = gb.predict(test)
print("Predictions have finished")

print(accuracy_score(target_val, gb_predictions))
print(classification_report(target_val, gb_predictions))

Predictions have finished
0.9715447154471545
              precision    recall  f1-score   support

           0       0.14      0.50      0.22         2
           1       1.00      0.98      0.99       244

    accuracy                           0.97       246
   macro avg       0.57      0.74      0.60       246
weighted avg       0.99      0.97      0.98       246



In [26]:
### classifier models
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('Naive Bayes',GaussianNB()))
models.append(('GradientBoost',GradientBoostingClassifier()))
models.append(('RandomForest', RandomForestClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier(n_neighbors = 5)))

In [27]:
# Make predictions on validation dataset

for name, model in models:
    print(name)
    model.fit(smote_train, smote_target)
    
    # Make predictions.
    predictions = model.predict(test)

    # Compute the error.
#     from sklearn.metrics import confusion_matrix
    print(classification_report(target_val, predictions))

    from sklearn.metrics import accuracy_score
    print(accuracy_score(target_val, predictions))
    print('\n')

LogisticRegression
              precision    recall  f1-score   support

           0       0.01      1.00      0.02         2
           1       1.00      0.12      0.22       244

    accuracy                           0.13       246
   macro avg       0.50      0.56      0.12       246
weighted avg       0.99      0.13      0.22       246

0.13008130081300814


Naive Bayes
              precision    recall  f1-score   support

           0       0.01      0.50      0.01         2
           1       0.99      0.41      0.58       244

    accuracy                           0.41       246
   macro avg       0.50      0.45      0.29       246
weighted avg       0.98      0.41      0.57       246

0.4065040650406504


GradientBoost
              precision    recall  f1-score   support

           0       0.14      0.50      0.22         2
           1       1.00      0.98      0.99       244

    accuracy                           0.97       246
   macro avg       0.57      0.74      0

In [28]:
# highest accuracy score is with RandomForest
final_test_df = test_df1.drop('Emp_ID', axis=1)
val_pred = gb.predict(final_test_df)
val_pred[:5]

array([1, 0, 1, 1, 1])

In [29]:
test_df['Target'] = val_pred
print(test_df.shape)
test_df.head()

(741, 2)


,Emp_ID,Target
0,394,1
1,173,0
2,1090,1
3,840,1
4,308,1


In [30]:
test_df.Target.value_counts()

1    722
0     19
Name: Target, dtype: int64

In [31]:
test_df.to_csv('Emp_Attr_V1.csv', index=None)

### --------- Now Folloing the same attr and method but on full dataset (including test datast emp_id) -------

In [32]:
final_df1.Attrition.value_counts() / len(final_df1)

1    0.678706
0    0.321294
Name: Attrition, dtype: float64

In [33]:
X = final_df1.drop(['Attrition', 'Emp_ID'], axis=1)
y = final_df1[['Attrition']]

# Split data into train and test sets as well as for validation and testing
train, test, target_train, target_val = train_test_split(X, y, train_size= 0.80, random_state=0)

oversampler=SMOTE(random_state=0)
smote_train, smote_target = oversampler.fit_resample(train, target_train)

### classifier models
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('Naive Bayes',GaussianNB()))
models.append(('GradientBoost',GradientBoostingClassifier()))
models.append(('RandomForest', RandomForestClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier(n_neighbors = 5)))

# Make predictions on validation dataset

for name, model in models:
    print(name)
    model.fit(smote_train, smote_target)
    
    # Make predictions.
    predictions = model.predict(test)

    # Compute the error.
#     from sklearn.metrics import confusion_matrix
    print(classification_report(target_val, predictions))

    from sklearn.metrics import accuracy_score
    print(accuracy_score(target_val, predictions))
    print('\n')

LogisticRegression
              precision    recall  f1-score   support

           0       0.54      0.52      0.53       168
           1       0.74      0.75      0.75       309

    accuracy                           0.67       477
   macro avg       0.64      0.64      0.64       477
weighted avg       0.67      0.67      0.67       477

0.6729559748427673


Naive Bayes
              precision    recall  f1-score   support

           0       0.77      0.39      0.52       168
           1       0.74      0.94      0.83       309

    accuracy                           0.74       477
   macro avg       0.75      0.66      0.67       477
weighted avg       0.75      0.74      0.72       477

0.7442348008385744


GradientBoost
              precision    recall  f1-score   support

           0       0.77      0.80      0.79       168
           1       0.89      0.87      0.88       309

    accuracy                           0.85       477
   macro avg       0.83      0.84      0.

In [34]:
rf = RandomForestClassifier() #
rf.fit(smote_train, smote_target)
print("Fitting of Random Forest finished")

rf_predictions = rf.predict(test)
print("Predictions finished")

''' Scoring the model '''
print("Accuracy score: {}".format(accuracy_score(target_val, rf_predictions)))
print("="*80)
print(classification_report(target_val, rf_predictions))

Fitting of Random Forest finished
Predictions finished
Accuracy score: 0.8553459119496856
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       168
           1       0.90      0.88      0.89       309

    accuracy                           0.86       477
   macro avg       0.84      0.85      0.84       477
weighted avg       0.86      0.86      0.86       477



In [35]:
# highest accuracy score is with RandomForest and GB
final_test_df = test_df1.drop('Emp_ID', axis=1)
val_pred = rf.predict(final_test_df)
test_df.drop('Target', axis=1, inplace=True)
test_df['Target'] = val_pred

test_df.to_csv('Emp_Attr_V2.csv', index=None)

print(test_df.shape)
test_df.head()

(741, 2)


,Emp_ID,Target
0,394,0
1,173,0
2,1090,0
3,840,0
4,308,0


In [36]:
test_df.Target.value_counts()

0    716
1     25
Name: Target, dtype: int64

### ----- Now Without SMOTE and on Overall data (including test employee) ------

In [37]:
X = final_df1.drop(['Attrition', 'Emp_ID'], axis=1)
y = final_df1[['Attrition']]

# Split data into train and test sets as well as for validation and testing
train, test, target_train, target_val = train_test_split(X, y, train_size= 0.80, random_state=0)

# oversampler=SMOTE(random_state=0)
# smote_train, smote_target = oversampler.fit_resample(train, target_train)

### classifier models
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('Naive Bayes',GaussianNB()))
models.append(('GradientBoost',GradientBoostingClassifier()))
models.append(('RandomForest', RandomForestClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier(n_neighbors = 5)))


# Make predictions on validation dataset

for name, model in models:
    print(name)
    model.fit(train, target_train)
    
    # Make predictions.
    predictions = model.predict(test)

    # Compute the error.
#     from sklearn.metrics import confusion_matrix
    print(classification_report(target_val, predictions))

    from sklearn.metrics import accuracy_score
    print(accuracy_score(target_val, predictions))
    print('\n')

LogisticRegression
              precision    recall  f1-score   support

           0       0.77      0.37      0.50       168
           1       0.73      0.94      0.82       309

    accuracy                           0.74       477
   macro avg       0.75      0.65      0.66       477
weighted avg       0.74      0.74      0.71       477

0.7379454926624738


Naive Bayes
              precision    recall  f1-score   support

           0       0.82      0.33      0.47       168
           1       0.73      0.96      0.83       309

    accuracy                           0.74       477
   macro avg       0.77      0.65      0.65       477
weighted avg       0.76      0.74      0.70       477

0.740041928721174


GradientBoost
              precision    recall  f1-score   support

           0       0.81      0.78      0.80       168
           1       0.88      0.90      0.89       309

    accuracy                           0.86       477
   macro avg       0.85      0.84      0.8

In [38]:
gb = GradientBoostingClassifier() #
gb.fit(train, target_train)
print("Fitting of Random Forest finished")

gb_predictions = gb.predict(test)
print("Predictions finished")

''' Scoring the model '''
print("Accuracy score: {}".format(accuracy_score(target_val, gb_predictions)))
print("="*80)
print(classification_report(target_val, gb_predictions))

Fitting of Random Forest finished
Predictions finished
Accuracy score: 0.859538784067086
              precision    recall  f1-score   support

           0       0.81      0.78      0.80       168
           1       0.88      0.90      0.89       309

    accuracy                           0.86       477
   macro avg       0.85      0.84      0.84       477
weighted avg       0.86      0.86      0.86       477



In [39]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
# print(cross_val_score(svm.SVC(kernel='rbf', gamma=0.7, C = 1.0), X, y, 
#                 scoring=make_scorer(f1_score, average='weighted', labels=[2]), cv=10))
print(cross_val_score(gb, X, y, cv=10, scoring='accuracy').mean())

0.863512183115924


In [40]:
# highest accuracy score is with  GB
final_test_df = test_df1.drop('Emp_ID', axis=1)
val_pred = gb.predict(final_test_df)
test_df.drop('Target', axis=1, inplace=True)
test_df['Target'] = val_pred

test_df.to_csv('Emp_Attr_V3.csv', index=None)

print(test_df.shape)
print(test_df.head(3))
test_df.Target.value_counts()

(741, 2)
   Emp_ID  Target
0     394       0
1     173       0
2    1090       0


0    649
1     92
Name: Target, dtype: int64

In [41]:
# checking V3 with  RF
rf = RandomForestClassifier() #
rf.fit(train, target_train)
print("Fitting of Random Forest finished")

rf_predictions = rf.predict(test)
final_test_df = test_df1.drop('Emp_ID', axis=1)
val_pred = rf.predict(final_test_df)
test_df.drop('Target', axis=1, inplace=True)
test_df['Target'] = val_pred

test_df.to_csv('Emp_Attr_V4.csv', index=None)

print(test_df.shape)
print(test_df.head(3))
test_df.Target.value_counts()

Fitting of Random Forest finished
(741, 2)
   Emp_ID  Target
0     394       0
1     173       0
2    1090       0


0    701
1     40
Name: Target, dtype: int64

In [42]:


X = final_train_df.drop(['Attrition'], axis=1)
y = final_train_df[['Attrition']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
log_reg=LogisticRegression(C=1000, max_iter=10000)
log_reg.fit(X_train, y_train)
pred = log_reg.predict(X_test)

print('--------------------------------------------------------------------------')
print('Logistic Regression:')
print('Traning Model accruracy scores: {:.3f}'.format(log_reg.score(X_train,y_train)))
print('Test Model accruracy scores: {:.3f}'.format(log_reg.score(X_test,y_test)))
print('Compute the error: \n', confusion_matrix(pred, y_test))
print('--------------------------------------------------------------------------')

--------------------------------------------------------------------------
Logistic Regression:
Traning Model accruracy scores: 0.986
Test Model accruracy scores: 0.983
Compute the error: 
 [[  0   0]
 [  7 403]]
--------------------------------------------------------------------------


In [43]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(log_reg, X, y, cv=5, scoring ='accuracy').mean())

0.9853658536585366
